# Optimización de hiper-parámetros - Avanzado

En este notebook jugaremos con dos tipos de optimización de hiperparámetros más complicados:
* Hyperopt
* Algoritmos genéticos


## Hyper-opt

(Hyperopt: Distributed Asynchronous Hyper-parameter Optimization, https://github.com/hyperopt/hyperopt).

Hyperopt es una librería escrita en Python que permite optimizar funciones de una forma rápida fijándose más en los valores que más probablemente van a dar una buena solución.

Actualmente tiene dos algoritmos implementados para hacer esto:

* Random Search
* Tree of Parzen Estimators (TPE)

Además, se pueden ejecutar en serie o en paralelo, haciendo uso de MongoDB.

Vamos a ver un ejemplo de cómo utilizarlo.

Vamos a encontrar el mínimo de $x^2$:
(<a href="https://www.google.es/search?source=hp&ei=2ds5XO_jG6y_lwTvxp2QBQ&q=x**2&btnK=Google+Search&oq=x**2&gs_l=psy-ab.3...716.1584..1916...0.0..0.301.1084.1j0j3j1......0....1..gws-wiz.....0..0i131j0.cgRgMW95cQk">click aqui para ver la funcion</a>)


In [ ]:
from hyperopt import fmin, tpe, hp
# con 10 iteraciones

best = fmin(fn=lambda x: x ** 2,
            space=hp.uniform('x', -10, 10),
            algo=tpe.suggest,
            max_evals=10)

print(best)

100%|██████████| 10/10 [00:00<00:00, 537.43trial/s, best loss: 2.212401117457257]
{'x': 1.4874142386898335}


In [ ]:
# con 100 iteraciones
best = fmin(fn=lambda x: x ** 2,
            space=hp.uniform('x', -10, 10),
            algo=tpe.suggest,
            max_evals=100)

print(best)

100%|██████████| 100/100 [00:00<00:00, 304.35trial/s, best loss: 7.987245874356897e-05]
{'x': 0.00893713929306067}


In [ ]:
# con 1000 iteraciones
best = fmin(fn=lambda x: x ** 2,
            space=hp.uniform('x', -10, 10),
            algo=tpe.suggest,
            max_evals=1000)

print(best)

100%|██████████| 1000/1000 [00:15<00:00, 66.28trial/s, best loss: 1.319331445529426e-06]
{'x': 0.0011486215414702207}


Y ahora uno más complejo con redes neuronales:

In [ ]:
# instalamos los paquetes necesarios
!pip install networkx
!pip install hyperopt

In [ ]:
# imports necesarios
import sys
import time
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import max_norm
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.datasets import cifar10

SEED = 42

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
validation_split = 0.1
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_split, random_state=SEED)

# normalizamos
X_train = X_train.astype('float32') / 255.
X_val = X_val.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# convertimos las etiquetas a one-hot encoding
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_val = to_categorical(y_val, n_classes)
y_test = to_categorical(y_test, n_classes)

# definimos nuestro espacio de búsqueda
# vamos a variar:
# - el número de filtros en nuestras capas conv
# - el porcentaje de dropout
# - el número de neuronas en la capa dense
space = {
    'n_filters_conv': hp.choice('n_filters_conv', [32, 64, 128]),
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'neurons_dense': hp.choice('neurons_dense', [256, 512, 1024]),
}

def	get_callbacks(pars):
  callbacks	= [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0, mode='auto')]
  return callbacks

def mi_cnn(pars):
  print ('Parameters: ', pars)
  model = Sequential()

  # Primer bloque convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Segundo bloque convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Tercer bloque convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Bloque clasificador
  model.add(Flatten())
  model.add(Dense(pars['neurons_dense'], activation='relu', kernel_constraint=max_norm(3.)))
  model.add(Dropout(pars['dropout']))
  model.add(Dense(10, activation='softmax'))

  # Compilamos el modelo
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=0.0001, decay=1e-6),
                metrics=['accuracy'])

  # Entrenamos el modelo
  history = model.fit(X_train,
                      y_train,
                      batch_size=128,
                      shuffle=True,
                      epochs=5,
                      validation_data=(X_val, y_val),
                      verbose = 0,
                      callbacks = get_callbacks(pars))

  best_epoch_loss = np.argmin(history.history['val_loss'])
  best_val_loss = np.min(history.history['val_loss'])
  best_val_acc = np.max(history.history['val_accuracy'])

  print('Epoch {} - val acc: {} - val loss: {}'.format(best_epoch_loss, best_val_acc, best_val_loss))
  sys.stdout.flush()

  return {'loss': best_val_loss, 'best_epoch': best_epoch_loss, 'eval_time': time.time(), 'status': STATUS_OK, 'model': model, 'history': history}


trials = Trials()
best = fmin(mi_cnn, space, algo=tpe.suggest, max_evals=10, trials=trials)
print(best)

170498071/170498071 [==============================] - 16s 0us/step
Parameters: 
{'dropout': 0.17853520908195575, 'n_filters_conv': 32, 'neurons_dense': 256}
Epoch 4 - val acc: 0.39640000462532043 - val loss: 1.6586565971374512
Parameters: 
{'dropout': 0.3859553888023874, 'n_filters_conv': 64, 'neurons_dense': 512}
Epoch 4 - val acc: 0.42579999566078186 - val loss: 1.5929936170578003
Parameters: 
{'dropout': 0.40103881173321493, 'n_filters_conv': 64, 'neurons_dense': 512}
Epoch 4 - val acc: 0.40939998626708984 - val loss: 1.6302917003631592
Parameters: 
{'dropout': 0.07966907270916024, 'n_filters_conv': 128, 'neurons_dense': 1024}
Epoch 4 - val acc: 0.5569999814033508 - val loss: 1.2723073959350586
Parameters: 
{'dropout': 0.15484133742576117, 'n_filters_conv': 64, 'neurons_dense': 256}
Epoch 4 - val acc: 0.46939998865127563 - val loss: 1.463975191116333
Parameters: 
{'dropout': 0.21836569061993982, 'n_filters_conv': 64, 'neurons_dense': 256}
Epoch 4 - val acc: 0.45899999141693115 - va

In [ ]:
trials.trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 1.6586565971374512,
   'best_epoch': 4,
   'eval_time': 1712934768.4170504,
   'status': 'ok',
   'model': <keras.src.engine.sequential.Sequential at 0x79cbf9a68940>,
   'history': <keras.src.callbacks.History at 0x79cbf57414b0>},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'dropout': [0], 'n_filters_conv': [0], 'neurons_dense': [0]},
   'vals': {'dropout': [0.17853520908195575],
    'n_filters_conv': [0],
    'neurons_dense': [0]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2024, 4, 12, 15, 12, 24, 520000),
  'refresh_time': datetime.datetime(2024, 4, 12, 15, 12, 48, 417000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 1.5929936170578003,
   'best_epoch': 4,
   'eval_time': 1712934791.1628962,
   'status': 'ok',
   'model': <keras.src.engine.sequential.Sequential at 0x79cbefe2cfa0>,
   'history': <ker

¿Por qué dice que best_epoch=4? Porque history está indexado en 0.

In [ ]:
trials.trials[6]

{'state': 2,
 'tid': 6,
 'spec': None,
 'result': {'loss': 1.2798274755477905,
  'best_epoch': 4,
  'eval_time': 1712934931.3656561,
  'status': 'ok',
  'model': <keras.src.engine.sequential.Sequential at 0x79cbf00ad750>,
  'history': <keras.src.callbacks.History at 0x79cbec3659f0>},
 'misc': {'tid': 6,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'dropout': [6], 'n_filters_conv': [6], 'neurons_dense': [6]},
  'vals': {'dropout': [0.1950535557094324],
   'n_filters_conv': [2],
   'neurons_dense': [2]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2024, 4, 12, 15, 14, 48, 227000),
 'refresh_time': datetime.datetime(2024, 4, 12, 15, 15, 31, 365000)}

In [ ]:
trials.results

[{'loss': 1.6586565971374512,
  'best_epoch': 4,
  'eval_time': 1712934768.4170504,
  'status': 'ok',
  'model': <keras.src.engine.sequential.Sequential at 0x79cbf9a68940>,
  'history': <keras.src.callbacks.History at 0x79cbf57414b0>},
 {'loss': 1.5929936170578003,
  'best_epoch': 4,
  'eval_time': 1712934791.1628962,
  'status': 'ok',
  'model': <keras.src.engine.sequential.Sequential at 0x79cbefe2cfa0>,
  'history': <keras.src.callbacks.History at 0x79cbec500340>},
 {'loss': 1.6302917003631592,
  'best_epoch': 4,
  'eval_time': 1712934813.9101667,
  'status': 'ok',
  'model': <keras.src.engine.sequential.Sequential at 0x79cbec45f040>,
  'history': <keras.src.callbacks.History at 0x79cbf0f4fbe0>},
 {'loss': 1.2723073959350586,
  'best_epoch': 4,
  'eval_time': 1712934842.7099383,
  'status': 'ok',
  'model': <keras.src.engine.sequential.Sequential at 0x79cbf05a96f0>,
  'history': <keras.src.callbacks.History at 0x79cbf0508d90>},
 {'loss': 1.463975191116333,
  'best_epoch': 4,
  'eval_

In [ ]:
trials.losses()

[1.6586565971374512,
 1.5929936170578003,
 1.6302917003631592,
 1.2723073959350586,
 1.463975191116333,
 1.50393545627594,
 1.2798274755477905,
 1.6552871465682983,
 1.5022732019424438,
 1.3085843324661255]

Qué os parece? mola eh? Así, podéis dejar vuestra configuración e iros a hacer algo más útil que andar variando parámetros hasta que encontréis la configuración adecuada.

**Mejor que lo hagan por nosotros y que nos la encontremos automáticamente seleccionada!**

Pero no tenemos por qué quedarnos aquí, podemos también variar el número de capas o si queremos conexiones residuales, por ejemplo! Sí, esto significa que podemos variar la **arquitectura** también!!

Aquí tenéis un ejemplo muy completo: https://github.com/Vooban/Hyperopt-Keras-CNN-CIFAR-100

Y aquí otro que quizá os parezca interesante:

<img src="http://cdn.lizamayliza.com/storage/cache/images/003/636/Daffy-Duck-Money-eyes-feature,xlarge.1480369578.jpg" border="0" height="200">

https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-hyperparameters-optimization-cb2b4a29b8ee

## 6.3 Algoritmos genéticos

Quizás estéis pensando en que tras haber visto esto no necesitáis saber nada más, ya habéis encontrado algo que es la repera! Pues atentos porque lo que viene, pese a ser algo muy sencillo, es potentísimo, y os va encantar. Sí, estoy hablando de **los algoritmos genéticos**.

En esencia, los algoritmos genéticos son un método de búsqueda meta-heurística inspirados en la evolución natural. Pertenecen a los algoritmos evolutivos, concretamente a los Algoritmos de búsqueda aleatoria guiada (Guided Random Search algorithms (Evolutionary Alg.)).

Esto que os puede parecer chino, es muy sencillo. Vamos a entenderlos con un ejemplo:

<img src="https://image.ibb.co/cJcQYJ/ga_problem.png" alt="ga_problem" border="0">

Imaginad que tenemos un puzzle, y nos queda solo una pieza por encajar. Lo que pasa es que este puzzle es muy especial, porque nos permite fabricarnos nuestras propias piezas. Para ello, disponemos de varios mecanismos:

* **combinar** partes de piezas (**crossover** o recombinación)
* **modificar** determinadas partes de esas piezas (**mutación**)
* **escoger** las mejores piezas de todas las que hemos hecho, para a partir de ellas, construir nuevas que sean mejores (**selección**)

Entonces imaginaos que decidimos cortar 10 trozos de carton, que son nuestras 10 piezas iniciales con las que vamos a probar a ver si alguna encaja perfectamnte. Las probamos todas, y de esas 10, 5 encajan más o menos. Así que seleccionamos esas 5 y fabricamos nuevas a partir de ellas mediante los mecanismos explicados arriba:

* de las 5 seleccionadas, sacamos 5 más combinando partes de dos de esas 5 originales escogiéndolas aleatoriamente
* de las 5 originales, y las nuevas 5 que nos hemos creado, sacamos 5 más modificando ligeramente una de las puntas de la pieza

Ahora resulta que tenemos 15 piezas, y nosotros queremos tener siempre 10, porque si no a la 5 vez que hiciéramos esto tendríamos una barbaridad de piezas!! Así que:

* probamos nuestras 15 piezas y nos quedamos con la que mejor encaja, y luego 9 escogidas al azar

Pues ya está, eso es lo que hace un algoritmo genético!! Ya sabéis cómo funcionan!! Sencillo, verdad? Pues no os hacéis una idea de lo potentes que son :-)

Vamos a verlo un poco más concreto siguiendo con nuestro ejemplo:

<img src="https://image.ibb.co/b7ySfy/geneticos_puzzle.png" alt="geneticos_puzzle" border="0">

Como podéis ver:

* cada pieza de nuestro conjunto de piezas (población) es un **cromosoma**
* cada parte de nuestra pieza es un **gen**, por tanto, nuestro cromosoma tiene 4 genes
* los posibles valores o configuraciones que puede tener cada gen se llama **alelo**

Esto es exactamente igual que en biología! No os dije que estaban inspirados en la evolución natural? :-)

Vale, pues vamos a ir relacionando estas palabrejas con nuestro ejemplo:

* necesitamos encontrar la pieza correcta para nuestro hueco del puzzle
* tenemos un conjunto inicial de piezas (**población**) que pueden encajar bien o no, no lo sabemos
* comprobamos cómo de bien encajan estas piezas (usando la **función de fitness**)
* si ninguna de las piezas encaja como nos gustaría, modificamos las piezas (con los operadores: **crossover** y **mutación**)
* comprobamos como de bien encajan las piezas recien creadas (**función de fitness**)
* seleccionamos las piezas que queremos aguantar para la próxima iteración (**selección**)

* y volvemos a empezar! así, hasta que enocntremos una pieza que encaje con la precisión que nosotros queremos

Venga, vamos a ponernos un poco más serios. Veamos el pseudo-algoritmo:

**START**

Generate the initial population

Compute fitness

**REPEAT**
    
    Selection
    
    Crossover
    
    Mutation

    Compute fitness

**UNTIL** population has converged

**STOP**

Se entiende, no? Es exactamente lo mismo que los pasos que acabamos de hablar con el puzzle!!

<img src="https://image.ibb.co/kQu7Ed/ga_workflow.png" alt="ga_workflow" border="0" width="600">

Vale, y ahora llega lo interesante, cómo funcionan realmente?

Tenemos que entender varios conceptos:

* cómo se inicializa nuestra población
* cómo funciona el crossover
* cómo funciona la mutación
* cómo funciona la seleccion
* cómo podemos definir nuestra función de fitness

Preparados? Vamos allá!

Lo primero es entender que cuando tenemos un problema en el mundo real y queremos solucionarlo en un ordenador, necesitamos **codificarlo** para que el ordenador lo entienda.

Así, por ejemplo:

* En el mundo real, el cromosoma es la pieza del puzzle. En el ordenador, el cromosoma es un vector con 4 valores (uno para indicar el tamaño de cada punta, donde positivo significa punta, negativo significa hueco hacia dentro de la pieza)

Esto es a lo que se llama la codificación.

Una vez sabido esto, vamos a ver cómo funcionan los operadores. Antes de nada, debéis saber que xisten muchos tipos de crossover, mutación y selección, pero aquí vamos a ver solo los más sencillos por temas de tiempo.

Si estáis interesados en conocerlos más profundamente, en internet hay muchísimo material disponible. Podéis empezar aquí: https://www.tutorialspoint.com/genetic_algorithms/index.htm

Vale, al turrón!

<img src="https://image.ibb.co/hEwVHd/ga_operators.png" alt="ga_operators" border="0">

**El crossover de un único punto (single-point crossover)**

Nuestro cromosoma es la pieza de puzzle, que tiene los 4 genes que veis en la imagen. Pues el crossover simple sencillamente escoge un punto aleatoriamente de los 4 genes, y combina las partes en nuevos cromosomas, como veis en la imagen.

Es importante que entendáis que el crossover **produce nuevos cromosomas**, puesto que tenemos los originales y los **recombinados**.

**La mutación uniforme**

La mutación uniforme consiste en que para cada cromosoma, lanzamos una moneda al aire. Si sale cara, modificamos un gen escogido aleatoriamente. ¿Qué valor le asignamos? Uno aleatorio dentro del rango que permite dicho gen.

**La selección**

Para la selección lo que se suele hacer es usar las fitness de los cromosomas (también llamados posibles **soluciones**). En este caso, vamos a ver la Stochastic Universal Sampling, que consiste en que construímos una gráfica de tipo tarta donde cada cromosoma ocupa un espacio que corresponde con su fitness. Después, establecemos N puntos fijos alrededor de la "tarta", donde N son el número de cromomsomas que queremos **seleccionar**. Después, se "gira la tarta", como si fuese la ruleta de la suerte, y los cromosomas a los que apuntan los puntos fijos son los seleccionados y pasan a la siguiente iteración.

Si os fijáis, **los cromosomas no están ordenados de mayor a menor fitness**!!

Esto es importante, puesto que si no, las probabilidades de seleccionar un cromosoma con una fitness alta y otro con una fitness baja serían más altas que de seleccionar dos con la fitness alta, ya que al estar los puntos de selección uno enfrente del otro, sería muy complicado seleccionar dos cromosomas con fitness parecidas.

Este operador tiene varias formas de funcionar. Siguiendo con nuestro ejemplo de población de 10 cromosomas, las formas son:

* Seleccionamos $N=10$ cromosomas, es decir, sustituimos la anterior población por una completamente nueva
* Seleccionamos $N=n$ cromosomas, donde n<10. Es decir, sustituímos solo una parte de los crosomomas antiguos. El resto siguen jugando ;-D

Vale, pues si seleccionamos los 10 está claro, pero si seleccionamos $n$, cómo elegimos cuales quitamos?

Pues las dos formas más comunes son:

* Quitamos los cromosomas más antiguos
* Quitamos los cromosomas con peor fitness

Ya por último, hay veces que seleccionamos al mejor cromosoma (o a los $k$ mejors) para que pase si o si a la siguiente iteración, es decir, hay *elitismo*. Hay que ir con cuidado con esto, puesto que aunque a priori parezca que el elitismo es lo mejor y que solo deberíamos quedarnos con los mejores, si lo hiciésemos nos estaríamos cargando una de las mayores virtudes de los genéticos: **que son capaces de escapar a mínimos locales!!!**

Fijaos, aquí podéis ver en plena acción a un genético intentando decidir cual es la mejor configuración para un vehículo de 2 ruedas: http://rednuht.org/genetic_cars_2/

Curioso, eh?

Vale, pues ahora qué tal si implementamos un par de ejemplos nosotros?





In [ ]:
# ejemplo simple de un GA donde tenemos que encontrar N números que sumen X
# https://lethain.com/genetic-algorithms-cool-name-damn-simple

from random import randint, random
from operator import add
from functools import reduce
import numpy as np

def individual(length, min, max):
    # creamos un individuo
    return [ randint(min,max) for x in range(length) ]

def population(count, length, min, max):
    # creamos nuestra población
    # count: numero de invidiuos en la población
    # length: número de valores por individuo
    # min: minimo permitido para cada valor del individuo
    # max: maximo permitido para cada valor del individuo

    return [ individual(length, min, max) for x in range(count) ]

def fitness(individual, target):
    # calculamos la fitness de un individuo: más bajo es mejor

    sum = reduce(add, individual, 0)
    return abs(target-sum)

def grade(pop, target):
    # calculamos la media de la población entera
    summed = reduce(add, (fitness(x, target) for x in pop))
    return summed / (len(pop) * 1.0)

def find_best_solution(pop, target):
    # encuentra la mejor solución en la población actual y la imprime
    res = [fitness(x, target) for x in pop]
    res_min = np.min(res)
    res_min_idx = np.where(res == res_min)[0]
    for n in res_min_idx:
        print('Individual: ', n, 'Valores: ', *pop[n], ' Result: ', np.sum(pop[n]), 'Target; ', target)
    return res_min

def evolve(pop, target, retain=0.2, random_select=0.05, mutate=0.01):
    graded = [ (fitness(x, target), x) for x in pop]
    graded = [ x[1] for x in sorted(graded)]
    retain_length = int(len(graded)*retain)
    parents = graded[:retain_length]

    # añadimos individuos aleatoriamente para promover la diversidad genética
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)

    # mutamos algunos
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            individual[pos_to_mutate] = randint(i_min, i_max)

    # reproducimos (crossover) nuestros cromosomas (individuals, soluciones)
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = round(len(male) / 2)
            child = male[:half] + female[half:]
            children.append(child)
    parents.extend(children)
    return parents

# ejecutamos el GA
generations = 200
target = 108
p_count = 20
i_length = 10
i_min = 0
i_max = 100
error_accepted = 1
print('Inicializamos la población con 20 individuos.')
p = population(p_count, i_length, i_min, i_max)
print('Calculamos el fitness de esos 20 individuos.')
fitness_history = [grade(p, target),]
print('El mejor individuo de la población inicial es:')
find_best_solution(p, target)

print('Comienza la búsqueda:')
for i in range(generations):
    p = evolve(p, target, retain=0.2, random_select=0.4, mutate=0.4)
    res = grade(p, target)
    fitness_history.append(res)

    res_min = find_best_solution(p, target)
    print('Generación: ', i, ' Fitness medio de todos los individuos de la población:', res)

    if res_min < error_accepted:
      break

Inicializamos la población con 20 individuos.
Calculamos el fitness de esos 20 individuos.
El mejor individuo de la población inicial es:
Individual:  13 Valores:  2 54 80 20 46 13 35 47 4 2  Result:  303 Target;  108
Comienza la búsqueda:
Individual:  9 Valores:  62 6 20 65 41 13 35 47 4 2  Result:  295 Target;  108
Generación:  0  Fitness medio de todos los individuos de la población: 302.85
Individual:  17 Valores:  55 20 1 83 24 13 35 47 4 2  Result:  284 Target;  108
Generación:  1  Fitness medio de todos los individuos de la población: 256.25
Individual:  0 Valores:  18 20 1 83 24 13 35 47 4 2  Result:  247 Target;  108
Generación:  2  Fitness medio de todos los individuos de la población: 262.9
Individual:  0 Valores:  18 20 1 83 24 13 35 47 4 2  Result:  247 Target;  108
Generación:  3  Fitness medio de todos los individuos de la población: 244.5
Individual:  0 Valores:  18 20 1 83 24 13 35 47 4 2  Result:  247 Target;  108
Individual:  15 Valores:  18 20 1 83 24 13 35 47 4 2  

In [1]:
# Ejemplo un poco más complejo de un GA que tiene que encontrar la cadena 'Hello Word!'
# http://www.obitko.com/tutorials/genetic-algorithms/ga-basic-description.php

import random

class GeneticAlgorithm(object):
    def __init__(self, genetics):
        self.genetics = genetics
        pass

    def run(self):
        population = self.genetics.initial()
        while True:
            fits_pops = [(self.genetics.fitness(ch),  ch) for ch in population]
            if self.genetics.check_stop(fits_pops): break
            population = self.next(fits_pops)
            pass
        return population

    def next(self, fits):
        parents_generator = self.genetics.parents(fits)
        size = len(fits)
        nexts = []
        while len(nexts) < size:
            parents = next(parents_generator)
            cross = random.random() < self.genetics.probability_crossover()
            children = self.genetics.crossover(parents) if cross else parents
            for ch in children:
                mutate = random.random() < self.genetics.probability_mutation()
                nexts.append(self.genetics.mutation(ch) if mutate else ch)
                pass
            pass
        return nexts[0:size]
    pass

class GeneticFunctions(object):
    def probability_crossover(self):
        # returns rate of occur crossover(0.0-1.0)
        return 1.0

    def probability_mutation(self):
        # returns rate of occur mutation(0.0-1.0)
        return 0.0

    def initial(self):
        # returns list of initial population
        return []

    def fitness(self, chromosome):
        # returns domain fitness value of chromosome
        return len(chromosome)

    def check_stop(self, fits_populations):
        # stop run if returns True
        # - fits_populations: list of (fitness_value, chromosome)
        return False

    def parents(self, fits_populations):
        r"""generator of selected parents
        """
        gen = iter(sorted(fits_populations))
        while True:
            f1, ch1 = next(gen)
            f2, ch2 = next(gen)
            yield (ch1, ch2)
            pass
        return

    def crossover(self, parents):
        r"""breed children
        """
        return parents

    def mutation(self, chromosome):
        r"""mutate chromosome
        """
        return chromosome
    pass

if __name__ == "__main__":
    """
    example: Mapped guess prepared Text
    """
    class GuessText(GeneticFunctions):
        def __init__(self, target_text,
                     limit=200, size=400,
                     prob_crossover=0.9, prob_mutation=0.2):
            self.target = self.text2chromo(target_text)
            self.counter = 0

            self.limit = limit
            self.size = size
            self.prob_crossover = prob_crossover
            self.prob_mutation = prob_mutation
            pass

        # GeneticFunctions interface impls
        def probability_crossover(self):
            return self.prob_crossover

        def probability_mutation(self):
            return self.prob_mutation

        def initial(self):
            return [self.random_chromo() for j in range(self.size)]

        def fitness(self, chromo):
            # larger is better, matched == 0
            return -sum(abs(c - t) for c, t in zip(chromo, self.target))

        def check_stop(self, fits_populations):
            self.counter += 1
            if self.counter % 1 == 0:
                best_match = list(sorted(fits_populations))[-1][1]
                fits = [f for f, ch in fits_populations]
                best = max(fits)
                worst = min(fits)
                ave = sum(fits) / len(fits)
                print(
                    "[G %3d] score=(%4d, %4d, %4d): %r" %
                    (self.counter, best, ave, worst,
                     self.chromo2text(best_match)))
                pass
            return self.counter >= self.limit

        def parents(self, fits_populations):
            while True:
                father = self.tournament(fits_populations)
                mother = self.tournament(fits_populations)
                yield (father, mother)
                pass
            pass

        def crossover(self, parents):
            father, mother = parents
            index1 = random.randint(1, len(self.target) - 2)
            index2 = random.randint(1, len(self.target) - 2)
            if index1 > index2: index1, index2 = index2, index1
            child1 = father[:index1] + mother[index1:index2] + father[index2:]
            child2 = mother[:index1] + father[index1:index2] + mother[index2:]
            return (child1, child2)

        def mutation(self, chromosome):
            index = random.randint(0, len(self.target) - 1)
            vary = random.randint(-5, 5)
            mutated = list(chromosome)
            mutated[index] += vary
            return mutated

        # internals
        def tournament(self, fits_populations):
            alicef, alice = self.select_random(fits_populations)
            bobf, bob = self.select_random(fits_populations)
            return alice if alicef > bobf else bob

        def select_random(self, fits_populations):
            return fits_populations[random.randint(0, len(fits_populations)-1)]

        def text2chromo(self, text):
            return [ord(ch) for ch in text]
        def chromo2text(self, chromo):
            return "".join(chr(max(1, min(ch, 255))) for ch in chromo)

        def random_chromo(self):
            return [random.randint(1, 255) for i in range(len(self.target))]
        pass

    GeneticAlgorithm(GuessText("Hello World!")).run()
    pass

[G   1] score=(-375, -866, -1333): '\x8akq»U+6=(l\\2'
[G   2] score=(-331, -786, -1160): '\x10ihbQ6:@\x7f^\x88c'
[G   3] score=(-331, -707, -1057): '\x10ihbQ6:@\x7f^\x88c'
[G   4] score=(-327, -636, -946): '\x0fihbV6:@\x7f^\x88c'
[G   5] score=(-323, -566, -900): '\x10s^H\x90\x1dB\x88w^\x88c'
[G   6] score=(-269, -504, -828): 'RQPr`\x1f\x18\x8c\x7f\x7fDB'
[G   7] score=(-206, -459, -808): 'RQPr`\x1fmhwRCB'
[G   8] score=(-178, -413, -614): 'TihbeHmh\x7f^>\x1d'
[G   9] score=(-151, -374, -562): 'Tihbe\nLx\x7f^>\x1d'
[G  10] score=(-147, -334, -515): 'Tilbe\nLx\x7f^>\x1d'
[G  11] score=(-137, -302, -473): 'Tijwk@Un\x7f^>\x1d'
[G  12] score=(-117, -269, -406): 'Td}uf\x1dVnw^;\x1d'
[G  13] score=(-135, -242, -375): 'Ishqk\x1f:ny\x89\x82\x17'
[G  14] score=(-124, -217, -321): 'Ishkf\x19UcwO\x82\x17'
[G  15] score=( -95, -194, -307): 'Iierj\x1dSnw\x84\x82\x1c'
[G  16] score=(-104, -175, -261): 'Tijbk\x1dLns^>\x1d'
[G  17] score=( -86, -157, -243): 'Ijeko#Unw\x86\x82\x1c'
[G  18] score=( -85,

Fijaos como es capaz de llegar a la cadena "Hello World!" tras unas pocas generaciones!

Esto también puede aplicarse a una red neuronal. Para ello, se puede hacer uso de una implementación disponible en Github: https://github.com/jliphard/DeepEvolve

Y os recomiendo que le echéis un ojo a estos dos enlaces en los que evolucionan una red neuronal con un GA. Aquí no nos da tiempo, pero son muy interesantes:

### Ejemplo GA para evolucionar NN:

* https://blog.coast.ai/lets-evolve-a-neural-network-with-a-genetic-algorithm-code-included-8809bece164
* https://github.com/harvitronix/neural-network-genetic-algorithm